In [1]:
file_date = '2021-03-21'

<div style="max-width:1400px;margin-center: auto">
<img src="images\constructor.png" width="600"/>
</div>

In [2]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local') \
    .appName("constructor") \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()
import os
os.chdir(os.getcwd())

In [3]:
from pyspark.sql.functions import col,current_timestamp,lit

In [4]:
constructors_schema = "constructorId INT, constructorRef INT, name STRING, nationality STRING, url STRING "

In [5]:
constructor_df = spark.read.json(f'raw files\\{file_date}\\constructors.json', schema = constructors_schema )
constructor_df.show(2)

+-------------+--------------+----------+-----------+--------------------+
|constructorId|constructorRef|      name|nationality|                 url|
+-------------+--------------+----------+-----------+--------------------+
|            1|          null|   McLaren|    British|http://en.wikiped...|
|            2|          null|BMW Sauber|     German|http://en.wikiped...|
+-------------+--------------+----------+-----------+--------------------+
only showing top 2 rows



In [6]:
# constructor_trans_df = constructor_df.select(col("constructorId").alias("constructor_id"),
#                                              col("constructorRef").alias("constructor_ref"),
#                                              col("name"),
#                                              col("nationality"))
# constructor_trans_df.show(2)

In [7]:
# constructor_drop_df = constructor_df.drop(constructor_df.url)
# constructor_drop_df = constructor_df.drop("url")
# constructor_drop_df = constructor_df.drop(constructor_df["url"])
constructor_drop_df = constructor_df.drop(col("url"))
constructor_drop_df.show(2)

+-------------+--------------+----------+-----------+
|constructorId|constructorRef|      name|nationality|
+-------------+--------------+----------+-----------+
|            1|          null|   McLaren|    British|
|            2|          null|BMW Sauber|     German|
+-------------+--------------+----------+-----------+
only showing top 2 rows



In [8]:
constructor_fnl_df = constructor_drop_df.withColumn("ingestion_date",current_timestamp()).\
                    withColumnRenamed("constructorId","constructor_id").\
                    withColumnRenamed("constructorRef","constructor_ref").\
                    withColumn('file_date',lit(file_date))

In [9]:
constructor_fnl_df.show(2)

+--------------+---------------+----------+-----------+--------------------+----------+
|constructor_id|constructor_ref|      name|nationality|      ingestion_date| file_date|
+--------------+---------------+----------+-----------+--------------------+----------+
|             1|           null|   McLaren|    British|2024-01-31 13:18:...|2021-03-21|
|             2|           null|BMW Sauber|     German|2024-01-31 13:18:...|2021-03-21|
+--------------+---------------+----------+-----------+--------------------+----------+
only showing top 2 rows



In [10]:
constructor_fnl_df.write.mode('overwrite').format('delta').saveAsTable('f1_processed.constructors')